# DeepGTA - Multi-Object Tracking Demo

This notebook demonstrates the DeepGTA pipeline for multi-object tracking.

**Pipeline:**
- YOLOv11x Detection with configurable class handling
- Deep-EIoU Online Tracking with ReID features
- GTA-Link Offline Refinement

[![GitHub](https://img.shields.io/badge/GitHub-DeepGTA-blue)](https://github.com/HiteshG/DeepGTA)

## 1. Installation

In [ ]:
# Clone DeepGTA repository
!git clone https://github.com/HiteshG/DeepGTA.git
%cd DeepGTA

# Install dependencies
!pip install -q -r requirements.txt
!pip install -q lap cython-bbox gdown

print("Installation complete!")

## 2. Configuration

Configure the pipeline for your use case. Below is an example for hockey tracking.

In [ ]:
from deepgta import DeepGTAConfig, DeepGTAPipeline

# Example: Hockey tracking configuration
config = DeepGTAConfig(
    # === Detection Settings ===
    yolo_weights="/content/HockeyAI_model_weight.pt",  # Upload your model
    class_names=[
        "Center Ice", "Faceoff", "Goalpost",
        "Goaltender", "Player", "Puck", "Referee"
    ],
    track_classes=[3, 4, 5, 6],   # Goaltender, Player, Puck, Referee
    special_classes=[5],           # Puck - max-conf only
    detection_conf_thresh=0.5,
    detection_iou_thresh=0.7,

    # === Tracking Settings ===
    with_reid=True,
    reid_model="/content/sports_model.pth.tar-60",  # Optional
    track_high_thresh=0.7,
    track_low_thresh=0.4,
    track_buffer=90,

    # === Refinement Settings ===
    use_refinement=True,
    split_eps=0.65,
    split_min_samples=15,
    merge_dist_thresh=0.35,

    # === Output Settings ===
    output_video=True,
    draw_tracks=True,
    draw_labels=True,
    verbose=True,
)

print("Configuration created!")
print(f"Tracking classes: {[config.class_names[i] for i in config.track_classes]}")
print(f"Special classes: {[config.class_names[i] for i in config.special_classes]}")

## 3. Upload Files

Upload your video and model weights.

In [ ]:
from google.colab import files

print("Upload your files:")
print("1. YOLO model weights (.pt)")
print("2. Input video (.mp4)")
print("3. (Optional) ReID model weights")

uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Uploaded: {filename}")

### (Optional) Download Default ReID Model

In [ ]:
# Download default sports ReID model
import gdown

reid_url = "https://drive.google.com/uc?id=14zzlm1nI9Ws_Il9RYNChwPC7Fsul7xwl"
reid_output = "/content/sports_model.pth.tar-60"

gdown.download(reid_url, reid_output, quiet=False)
print(f"ReID model downloaded to: {reid_output}")

## 3.5 Test Detection (Important!)

Before running the full pipeline, test that detection is working on a sample frame. This helps identify issues with:
- Model weights path
- Class indices configuration
- Confidence threshold settings

In [ ]:
from deepgta import test_detection_on_frame, test_detection_multiple_frames
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt

# Update these paths to match your uploaded files
video_path = "/content/input_video.mp4"  # Your video
config.yolo_weights = "/content/HockeyAI_model_weight.pt"  # Your model

# Test on a specific frame (try different frame numbers)
frame_to_test = 50  # Change this to test different frames

print(f"Testing detection on frame {frame_to_test}...")
print(f"Config: track_classes={config.track_classes}")
print(f"Config: detection_conf_thresh={config.detection_conf_thresh}")
print()

# Run detection test
frame, filtered_dets, raw_dets = test_detection_on_frame(
    config, 
    video_path, 
    frame_number=frame_to_test,
    save_path="/content/detection_test.jpg",
    show=False
)

# Display results
print(f"\n{'='*50}")
print("DETECTION TEST RESULTS")
print('='*50)
print(f"Raw detections (all classes): {len(raw_dets)}")
print(f"Filtered detections (tracked classes only): {len(filtered_dets)}")

if len(raw_dets) == 0:
    print("\n⚠️  WARNING: No detections found!")
    print("Check:")
    print("  1. Model weights path is correct")
    print("  2. Video path is correct")
    print("  3. Try lowering detection_conf_thresh (e.g., 0.25)")
elif len(filtered_dets) == 0:
    print("\n⚠️  WARNING: Detections found but all filtered out!")
    print("Check your track_classes indices match your model's class order")
    print(f"  Your track_classes: {config.track_classes}")
    print(f"  Detected classes: {set(d.class_id for d in raw_dets)}")
else:
    print("\n✅ Detection is working!")

# Display the annotated frame
print("\nDisplaying annotated frame...")
display(Image(filename="/content/detection_test.jpg"))

In [ ]:
# Optional: Test on multiple frames across the video
# This gives you a better sense of detection performance

import os
os.makedirs("/content/test_frames", exist_ok=True)

test_detection_multiple_frames(
    config,
    video_path,
    num_frames=5,  # Test 5 evenly-spaced frames
    save_dir="/content/test_frames"
)

# List saved test frames
print("\nSaved test frames:")
for f in sorted(os.listdir("/content/test_frames")):
    print(f"  /content/test_frames/{f}")

### Troubleshooting Detection Issues

**If no detections are found:**
1. Lower the confidence threshold: `config.detection_conf_thresh = 0.25`
2. Verify model weights path is correct
3. Check the video file is not corrupted

**If detections are filtered out:**
1. Print your model's class names to verify order
2. Adjust `track_classes` to match actual class indices
3. Run the cell below to see what classes are detected:

In [ ]:
# Troubleshooting: Check model class names and test with lower threshold
from ultralytics import YOLO

# Load model to check class names
model = YOLO(config.yolo_weights)
print("Model class names:")
for idx, name in model.names.items():
    print(f"  {idx}: {name}")

# Test with very low threshold to see all possible detections
print("\n" + "="*50)
print("Testing with LOW threshold (0.1) to see all detections:")
print("="*50)

# Temporarily lower threshold
original_thresh = config.detection_conf_thresh
config.detection_conf_thresh = 0.1

frame, filtered_dets, raw_dets = test_detection_on_frame(
    config, 
    video_path, 
    frame_number=frame_to_test,
    save_path="/content/detection_test_low_thresh.jpg",
    show=False
)

print(f"\nWith threshold=0.1: {len(raw_dets)} raw detections")
print("Detected class distribution:")
from collections import Counter
class_dist = Counter(d.class_id for d in raw_dets)
for cls_id, count in sorted(class_dist.items()):
    cls_name = config.class_names[cls_id] if cls_id < len(config.class_names) else f"class_{cls_id}"
    print(f"  Class {cls_id} ({cls_name}): {count} detections")

# Restore original threshold
config.detection_conf_thresh = original_thresh
print(f"\nRestored threshold to: {config.detection_conf_thresh}")

## 4. Run Pipeline

In [ ]:
# Update config with actual file paths
config.yolo_weights = "/content/HockeyAI_model_weight.pt"  # Update this
config.reid_model = "/content/sports_model.pth.tar-60"     # Update this

# Create pipeline
pipeline = DeepGTAPipeline(config)

# Process video
input_video = "/content/input_video.mp4"  # Update this
output_video = "/content/output_tracked.mp4"

output_path, tracklets = pipeline.process_video(
    video_path=input_video,
    output_path=output_video,
    show_progress=True
)

print(f"\nTracking complete!")
print(f"Output saved to: {output_path}")
print(f"Total unique tracks: {len(tracklets)}")

## 5. Display Results

In [ ]:
import subprocess
from IPython.display import HTML
from base64 import b64encode

# Compress video for display
compressed_video = "/content/output_compressed.mp4"
subprocess.run([
    "ffmpeg", "-i", output_video,
    "-vcodec", "libx264", "-crf", "28",
    "-preset", "fast",
    compressed_video, "-y"
], capture_output=True)

# Display in notebook
mp4 = open(compressed_video, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'''
<video width="800" controls>
    <source src="{data_url}" type="video/mp4">
</video>
''')

## 6. Analyze Results

In [ ]:
# Print tracking statistics
print("=" * 50)
print("Tracking Statistics")
print("=" * 50)

# Count tracks by class
class_counts = {}
total_detections = 0

for tid, tracklet in tracklets.items():
    class_id = tracklet.class_id
    class_name = config.class_names[class_id] if class_id < len(config.class_names) else f"class_{class_id}"

    if class_name not in class_counts:
        class_counts[class_name] = {'tracks': 0, 'detections': 0}

    class_counts[class_name]['tracks'] += 1
    class_counts[class_name]['detections'] += len(tracklet.times)
    total_detections += len(tracklet.times)

print(f"\nTotal unique tracks: {len(tracklets)}")
print(f"Total detections: {total_detections}")
print(f"\nBy class:")
for class_name, counts in sorted(class_counts.items()):
    print(f"  {class_name}: {counts['tracks']} tracks, {counts['detections']} detections")

# Track lengths
track_lengths = [len(t.times) for t in tracklets.values()]
print(f"\nTrack length statistics:")
print(f"  Min: {min(track_lengths)} frames")
print(f"  Max: {max(track_lengths)} frames")
print(f"  Mean: {sum(track_lengths) / len(track_lengths):.1f} frames")

## 7. Download Results

In [ ]:
from google.colab import files

# Download tracked video
files.download(output_video)

# Optionally download MOT format results
# if config.save_mot_txt:
#     files.download(config.mot_output_path)

---

## Custom Configuration Examples

### Soccer Tracking

In [ ]:
# Soccer tracking configuration
soccer_config = DeepGTAConfig(
    yolo_weights="/content/soccer_model.pt",
    class_names=["Background", "Player", "Ball", "Referee", "Goalkeeper"],
    track_classes=[1, 2, 3, 4],
    special_classes=[2],  # Ball - single instance
    detection_conf_thresh=0.4,

    # Longer buffer for soccer (players may be occluded longer)
    track_buffer=120,

    with_reid=True,
    use_refinement=True,
)

### Without Refinement (Faster Processing)

In [ ]:
# Fast mode - no refinement
fast_config = DeepGTAConfig(
    yolo_weights="/content/model.pt",
    class_names=["Person", "Car", "Truck"],
    track_classes=[0, 1, 2],

    # Disable refinement for faster processing
    use_refinement=False,

    # Disable ReID for even faster processing
    with_reid=False,

    output_video=True,
)